# Y1-A: Time-Control — Fixed λ=0.60, Long Training (57 epochs)\n\n**Purpose**: Test whether X3's degradation is explained by cumulative training time alone.\n\n- Clean start (fresh model + optimizer) per seed\n- λ = 0.60 fixed (same as X3's best region)\n- 57 epochs (= X3-fast total: 19 steps × 3 ep/step)\n- η = 0.8, r = 5%, SST-2, DistilBERT\n- Seeds: 20\n\n**Key comparison**: If Y1-A final error ≈ 0.18–0.30 → X3 trap is path-dependent. If Y1-A final error ≈ 0.63–0.70 → X3 trap is time-driven.

In [ ]:
!pip install -q transformers datasets accelerate

import os
import json
import time
import copy
import random
import warnings
import logging
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import (
    DistilBertTokenizer, DistilBertForSequenceClassification
)
from datasets import load_dataset

warnings.filterwarnings("ignore")
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu.name} ({gpu.total_mem/1e9:.1f} GB)" if hasattr(gpu, 'total_mem') else f"GPU: {gpu.name}")


## Config

In [ ]:
CONFIG = {
    "experiment": "exp_Y1A_time_control_long",
    "experiment_id": "Y1A",
    "series": "Y",
    "experiment_id_prefix": "Y1A",
    "dataset": "SST-2",
    "num_classes": 2,
    "model_name": "distilbert-base-uncased",
    "noise_type": "symmetric",
    "noise_rate": 0.8,
    "trusted_ratio": 0.05,
    "lambda_fixed": 0.60,
    "epochs": 57,  # matches X3-fast total (19 steps × 3 ep/step)
    "seeds": list(range(20)),
    "batch_size": 64,
    "learning_rate": 2e-5,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "max_seq_length": 128,
    "use_bf16": True,
    "max_train_samples": 16000,
    "output_dir": "Y1A_results",
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)

n_used = min(int(67349 * 0.95), CONFIG["max_train_samples"])
steps_per_epoch = n_used // CONFIG["batch_size"]
total_steps = steps_per_epoch * CONFIG["epochs"]
n_runs = len(CONFIG["seeds"])

print(f"Experiment: {CONFIG['experiment']}")
print(f"λ fixed: {CONFIG['lambda_fixed']}")
print(f"Epochs: {CONFIG['epochs']}")
print(f"Seeds: {n_runs}")
print(f"Steps/epoch: ~{steps_per_epoch}, Total steps/run: ~{total_steps}")
est_per_run = total_steps * 25 / 1000 + CONFIG["epochs"] * 5
print(f"Est. per run: ~{est_per_run/60:.1f} min")
print(f"Est. total: ~{n_runs * est_per_run / 3600:.1f} h")


## Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_BASE = "/content/drive/MyDrive/Paper-A_Y_series"
DRIVE_OUTPUT = f"{DRIVE_BASE}/exp_Y1A_time_control_long"
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
LOCAL_OUTPUT = CONFIG["output_dir"]

def save_to_drive(filename, data):
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        with open(os.path.join(d, filename), "w") as f:
            json.dump(data, f, indent=2)

def save_figure_to_drive(fig, filename, dpi=300):
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        fig.savefig(os.path.join(d, filename), dpi=dpi, bbox_inches="tight", facecolor="white")

print(f"Local: {LOCAL_OUTPUT}")
print(f"Drive: {DRIVE_OUTPUT}")


## Seed Function

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


## Load Data

In [ ]:
print("Loading SST-2...")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
raw_dataset = load_dataset("glue", "sst2")

train_texts = list(raw_dataset["train"]["sentence"])
train_labels = list(raw_dataset["train"]["label"])
test_texts = list(raw_dataset["validation"]["sentence"])
test_labels = list(raw_dataset["validation"]["label"])

print(f"Train: {len(train_texts)}, Test: {len(test_texts)}")

print("Tokenizing...")
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length",
                            max_length=CONFIG["max_seq_length"], return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding="max_length",
                           max_length=CONFIG["max_seq_length"], return_tensors="pt")
print("Done.")


In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

test_dataset = TextDataset(test_encodings, torch.tensor(test_labels, dtype=torch.long))
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
print(f"Test loader: {len(test_dataset)} samples")


## Noise Preparation

In [ ]:
def prepare_noisy_and_trusted(train_labels, noise_rate, trusted_ratio,
                              num_classes, seed, max_noisy_samples=None):
    rng = np.random.RandomState(seed + 10000)
    n = len(train_labels)
    labels = np.array(train_labels)
    indices = np.arange(n)

    trusted_indices = []
    for c in range(num_classes):
        class_idx = indices[labels == c]
        n_sel = max(1, int(len(class_idx) * trusted_ratio))
        trusted_indices.extend(rng.choice(class_idx, size=n_sel, replace=False))
    trusted_indices = np.array(sorted(trusted_indices))

    noisy_mask = np.ones(n, dtype=bool)
    noisy_mask[trusted_indices] = False
    noisy_indices = indices[noisy_mask]

    noisy_labels = labels.copy()
    n_flip = int(len(noisy_indices) * noise_rate)
    flip_idx = rng.choice(noisy_indices, size=n_flip, replace=False)
    for idx in flip_idx:
        candidates = [c for c in range(num_classes) if c != noisy_labels[idx]]
        noisy_labels[idx] = rng.choice(candidates)

    if max_noisy_samples and len(noisy_indices) > max_noisy_samples:
        noisy_indices = np.sort(rng.choice(noisy_indices, size=max_noisy_samples, replace=False))

    actual = np.mean(noisy_labels[noisy_indices] != labels[noisy_indices])
    print(f"  Trusted: {len(trusted_indices)} | Noisy: {len(noisy_indices)} (η={actual:.3f})")
    return torch.tensor(noisy_labels, dtype=torch.long), trusted_indices, noisy_indices

print("Noise test:")
_, ti, ni = prepare_noisy_and_trusted(train_labels, 0.8, 0.05, 2, 0, 16000)


## Core Functions

In [ ]:
def get_head_param_names(model):
    return [n for n, _ in model.named_parameters() if "classifier" in n or "pre_classifier" in n]

def extract_grads(model):
    return {n: p.grad.clone() for n, p in model.named_parameters() if p.grad is not None}

def normalize_grad_dict(grads):
    flat = torch.cat([g.flatten() for g in grads.values()])
    norm = flat.norm()
    if norm > 0:
        return {n: g / norm for n, g in grads.items()}, norm.item()
    return grads, 0.0

def cosine_sim(grads_a, grads_b, param_names=None):
    keys = [n for n in (param_names or sorted(set(grads_a) & set(grads_b)))
            if n in grads_a and n in grads_b]
    if not keys: return 0.0
    va = torch.cat([grads_a[n].flatten() for n in keys])
    vb = torch.cat([grads_b[n].flatten() for n in keys])
    return F.cosine_similarity(va.unsqueeze(0), vb.unsqueeze(0)).item()

def set_mixed_grad(model, g_struct_n, g_value_n, lam):
    for n, p in model.named_parameters():
        if n in g_struct_n and n in g_value_n:
            p.grad = (1 - lam) * g_struct_n[n] + lam * g_value_n[n]
        elif n in g_struct_n:
            p.grad = (1 - lam) * g_struct_n[n]
        elif n in g_value_n:
            p.grad = lam * g_value_n[n]

@torch.no_grad()
def evaluate(model, test_loader, device):
    model.eval()
    correct = total = 0
    total_loss = 0.0
    n = 0
    for batch in test_loader:
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        out = model(input_ids=ids, attention_mask=mask, labels=labels)
        preds = out.logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += out.loss.item()
        n += 1
    model.train()
    acc = correct / total
    return acc, 1.0 - acc, total_loss / n

print("Core functions defined.")


## Training Function

In [ ]:
def run_fixed_lambda(seed, config):
    """
    Execute one dual-gradient training run at fixed λ with epoch-level logging.
    Clean start (fresh model + optimizer).
    """
    lam = config["lambda_fixed"]
    exp_id = f"{config['experiment_id_prefix']}-{seed:03d}-lam{lam:.2f}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    set_seed(seed)
    t_start = time.time()

    # Data preparation
    noisy_labels, trusted_idx, noisy_idx = prepare_noisy_and_trusted(
        train_labels, config["noise_rate"], config["trusted_ratio"],
        config["num_classes"], seed, config.get("max_train_samples")
    )

    noisy_dataset = TextDataset(
        {k: v[noisy_idx] for k, v in train_encodings.items()},
        noisy_labels[noisy_idx]
    )
    trusted_dataset = TextDataset(
        {k: v[trusted_idx] for k, v in train_encodings.items()},
        noisy_labels[trusted_idx]
    )

    struct_loader = DataLoader(noisy_dataset, batch_size=config["batch_size"],
                               shuffle=True, drop_last=True)
    value_loader = DataLoader(trusted_dataset,
                              batch_size=min(config["batch_size"], len(trusted_dataset)),
                              shuffle=True, drop_last=False)

    # Model — fresh init
    set_seed(seed)
    model = DistilBertForSequenceClassification.from_pretrained(
        config["model_name"], num_labels=config["num_classes"]
    ).to(device)
    model.train()
    head_params = get_head_param_names(model)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    use_amp = config.get("use_bf16", False) and device.type == "cuda"
    amp_dtype = torch.bfloat16 if use_amp else torch.float32

    # Training
    epoch_logs = []
    global_step = 0
    best_error = 1.0
    best_epoch = 0

    for epoch in range(config["epochs"]):
        value_iter = iter(value_loader)
        epoch_loss_s = 0.0
        epoch_loss_v = 0.0
        epoch_cos_sv = []
        n_batches = 0

        for struct_batch in struct_loader:
            # Struct gradient
            optimizer.zero_grad()
            s_ids = struct_batch["input_ids"].to(device)
            s_mask = struct_batch["attention_mask"].to(device)
            s_labels = struct_batch["labels"].to(device)
            with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                out_s = model(input_ids=s_ids, attention_mask=s_mask, labels=s_labels)
            out_s.loss.backward()
            g_struct = extract_grads(model)

            # Value gradient
            optimizer.zero_grad()
            try:
                vb = next(value_iter)
            except StopIteration:
                value_iter = iter(value_loader)
                vb = next(value_iter)

            v_ids = vb["input_ids"].to(device)
            v_mask = vb["attention_mask"].to(device)
            v_labels = vb["labels"].to(device)
            with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                out_v = model(input_ids=v_ids, attention_mask=v_mask, labels=v_labels)
            out_v.loss.backward()
            g_value = extract_grads(model)

            # Normalize
            g_struct_n, norm_s = normalize_grad_dict(g_struct)
            g_value_n, norm_v = normalize_grad_dict(g_value)

            # Cosine similarity
            c_sv = cosine_sim(g_struct_n, g_value_n, head_params)
            epoch_cos_sv.append(c_sv)

            # Mixed gradient
            optimizer.zero_grad()
            set_mixed_grad(model, g_struct_n, g_value_n, lam)
            optimizer.step()

            epoch_loss_s += out_s.loss.item()
            epoch_loss_v += out_v.loss.item()
            n_batches += 1
            global_step += 1

        # End of epoch eval
        acc, error, test_loss = evaluate(model, test_loader, device)
        if error < best_error:
            best_error = error
            best_epoch = epoch + 1

        epoch_log = {
            "epoch": epoch + 1,
            "test_error": round(error, 6),
            "test_acc": round(acc, 6),
            "test_loss": round(test_loss, 6),
            "train_loss_struct": round(epoch_loss_s / max(n_batches, 1), 6),
            "train_loss_value": round(epoch_loss_v / max(n_batches, 1), 6),
            "mean_cos_sv": round(float(np.mean(epoch_cos_sv)), 6),
            "n_batches": n_batches
        }
        epoch_logs.append(epoch_log)

        if (epoch + 1) % max(1, config["epochs"] // 10) == 0 or epoch == 0:
            print(f"    Epoch {epoch+1}/{config['epochs']}: error={error:.4f} cos_sv={epoch_log['mean_cos_sv']:.4f}")

    elapsed = time.time() - t_start
    result = {
        "experiment_id": exp_id,
        "experiment": config["experiment"],
        "seed": seed,
        "lambda": lam,
        "noise_rate": config["noise_rate"],
        "trusted_ratio": config["trusted_ratio"],
        "epochs": config["epochs"],
        "test_error": round(epoch_logs[-1]["test_error"], 6),
        "best_error": round(best_error, 6),
        "best_epoch": best_epoch,
        "epoch_logs": epoch_logs,
        "time_seconds": round(elapsed, 1)
    }
    return result

print("Training function defined.")


## Execute

In [ ]:
all_results = []
n_runs = len(CONFIG["seeds"])
t_total = time.time()

print("=" * 70)
print(f"Y1-A: Time Control (Long) — λ={CONFIG['lambda_fixed']}, {CONFIG['epochs']} ep, {n_runs} seeds")
print("=" * 70)

for i, seed in enumerate(CONFIG["seeds"]):
    print(f"\nSeed {seed} ({i+1}/{n_runs})")
    print("-" * 50)
    result = run_fixed_lambda(seed, CONFIG)
    all_results.append(result)

    elapsed = time.time() - t_total
    eta = elapsed / (i+1) * (n_runs - i - 1)
    print(f"  Done: final={result['test_error']:.4f} best={result['best_error']:.4f}@ep{result['best_epoch']} "
          f"({result['time_seconds']:.0f}s) [ETA: {eta/60:.0f}m]")

    save_to_drive(f"Y1A_seed{seed:02d}.json", result)

total_time = time.time() - t_total
save_to_drive("Y1A_results.json", all_results)
save_to_drive("Y1A_config.json", CONFIG)

errors = [r["test_error"] for r in all_results]
bests = [r["best_error"] for r in all_results]
print(f"\n{'='*70}")
print(f"Y1-A COMPLETE: {n_runs} runs in {total_time/60:.1f} min")
print(f"Final error: {np.mean(errors):.4f} ± {np.std(errors):.4f}")
print(f"Best error:  {np.mean(bests):.4f} ± {np.std(bests):.4f}")
print(f"{'='*70}")


## Analysis

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

# Load results
path = os.path.join(LOCAL_OUTPUT, "Y1A_results.json")
if not os.path.exists(path):
    path = os.path.join(DRIVE_OUTPUT, "Y1A_results.json")
with open(path) as f:
    results = json.load(f)

n_ep = CONFIG["epochs"]

# Extract trajectories
trajs = np.array([[l["test_error"] for l in r["epoch_logs"]] for r in results])
cos_trajs = np.array([[l["mean_cos_sv"] for l in r["epoch_logs"]] for r in results])

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# (A) Error trajectory
ax = axes[0]
means = trajs.mean(axis=0)
stds = trajs.std(axis=0)
epochs = np.arange(1, n_ep + 1)
ax.plot(epochs, means, 'o-', color="#2CA02C", linewidth=2, markersize=3, label="Y1-A (λ=0.60, clean start)")
ax.fill_between(epochs, means - stds, means + stds, alpha=0.15, color="#2CA02C")

# X1b reference (λ=0.60, 3ep independent)
ax.axhline(0.185, color="#D62728", ls="--", lw=2, alpha=0.7, label="X1b (λ=0.60, 3ep) ≈ 18.5%")

# X3-fast reference (λ=0.60 after sweep)
ax.axhline(0.631, color="#FF7F0E", ls="--", lw=2, alpha=0.7, label="X3-fast (λ=0.60 after sweep) ≈ 63.1%")

ax.axhline(0.5, color="gray", ls=":", alpha=0.4)
ax.set_xlabel("Epoch", fontsize=13)
ax.set_ylabel("Test Error", fontsize=13)
ax.set_title("(A) Y1-A: Error Trajectory (λ=0.60 fixed)", fontsize=13, fontweight="bold")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.0)

# (B) Cosine similarity
ax = axes[1]
cos_means = cos_trajs.mean(axis=0)
cos_stds = cos_trajs.std(axis=0)
ax.plot(epochs, cos_means, 's-', color="#9467BD", linewidth=2, markersize=3)
ax.fill_between(epochs, cos_means - cos_stds, cos_means + cos_stds, alpha=0.15, color="#9467BD")
ax.set_xlabel("Epoch", fontsize=13)
ax.set_ylabel("cos(g_struct, g_value)", fontsize=13)
ax.set_title("(B) Gradient Alignment Over Time", fontsize=13, fontweight="bold")
ax.grid(True, alpha=0.3)

# (C) Final error distribution
ax = axes[2]
finals = [r["test_error"] for r in results]
ax.hist(finals, bins=15, color="#2CA02C", alpha=0.7, edgecolor="black")
ax.axvline(np.mean(finals), color="red", ls="--", lw=2, label=f"Mean: {np.mean(finals):.4f}")
ax.axvline(0.185, color="#D62728", ls=":", lw=2, alpha=0.7, label="X1b ref: 0.185")
ax.axvline(0.631, color="#FF7F0E", ls=":", lw=2, alpha=0.7, label="X3 ref: 0.631")
ax.set_xlabel("Final Test Error", fontsize=13)
ax.set_ylabel("Count", fontsize=13)
ax.set_title("(C) Distribution of Final Errors", fontsize=13, fontweight="bold")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_figure_to_drive(fig, "Y1A_analysis.png")
plt.show()

# Summary statistics
print("\n" + "=" * 60)
print("Y1-A Summary")
print("=" * 60)
print(f"Final error: {np.mean(finals):.4f} ± {np.std(finals):.4f}")
print(f"Best error:  {np.mean([r['best_error'] for r in results]):.4f}")
print(f"Best epoch:  {np.mean([r['best_epoch'] for r in results]):.1f}")
print(f"\nComparison:")
print(f"  X1b (λ=0.60, 3ep):    ~0.185")
print(f"  X3-fast (after sweep): ~0.631")
print(f"  Y1-A (57ep, clean):    {np.mean(finals):.4f}")
if np.mean(finals) < 0.35:
    print("\n→ CONCLUSION: X3 trap is PATH-DEPENDENT (not time-driven)")
    print("  Long training alone does NOT reproduce X3's degradation.")
elif np.mean(finals) > 0.55:
    print("\n→ CONCLUSION: X3 trap is TIME-DRIVEN (cumulative learning)")
    print("  Long training alone reproduces X3's degradation.")
else:
    print("\n→ CONCLUSION: MIXED — partial time effect + partial path effect")


## Save & Download

In [ ]:
print(f"All results saved to: {DRIVE_OUTPUT}")
for f in sorted(os.listdir(DRIVE_OUTPUT)):
    sz = os.path.getsize(os.path.join(DRIVE_OUTPUT, f))
    print(f"  {f} ({sz/1024:.1f} KB)")

import shutil
shutil.make_archive(CONFIG["output_dir"], "zip", LOCAL_OUTPUT)
try:
    from google.colab import files
    files.download(f"{CONFIG['output_dir']}.zip")
except:
    pass
